In [1]:
import time
start = time.time()

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import sqlite3

import nltk
#GitHub needs nltk.download()
import random

#NLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# pd.set_option('max_colwidth', 2000)
# pd.options.display.max_rows = 500
# pd.options.display.max_columns = 500

In [3]:
#Score predictions of fitted model.
def score_algo(model, X, y, prediction):
    from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
    print('-----------------------------------------------')
    print(f'##### {type(model).__name__} - Prediction Scores:')
    print('-----------------------------------------------')
    print('RMSE:', round(np.sqrt(mean_squared_error(y, prediction)), 1))
    print(' R^2:', round(r2_score(y, prediction), 4))
    print(' MAE:', round(mean_absolute_error(y, prediction),1))

In [4]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import JSON, JSONB
engine=create_engine("postgresql://postgres@localhost:5432/postgres")

In [5]:
#Read in data tables.  Put quotes around table to make case-sensitive.
vehicle=pd.read_sql('SELECT * from vehicle', engine, index_col='index')
train=pd.read_sql('SELECT * from "trainClean"', engine, index_col='index')
test=pd.read_sql('SELECT * from "testClean"', engine, index_col='index')
vehicle.index.name=None; train.index.name=None; test.index.name=None;

In [6]:
print('vehicle row count:', len(vehicle))
vehicle.head(1)

vehicle row count: 3514


,URL_Vehicle,Title,Location,Year,Year_in_Title,Odometer,RawMake,Make,Model,Trim,...,Body,AttribDictionary,ImageDictionary,VehicleID,VIN,TrimAlternate,DownFlag,Model_Trim,Model_Seller,Trim_Seller
1,https://austin.craigslist.org/cto/d/red-rock-2...,2006 Ford Focus SE,austin,2006,None,181000.0,2006 ford focus se,ford,focus,se,...,"\n ,\n2006 Ford Focus in great mechanic...",{'0': '<span><b>2006 Ford Focus SE</b></span>'...,{'0': 'https://images.craigslist.org/00n0n_dXT...,7022331083,None2,se,None,focus_se,focus_owner,se_owner


In [7]:
#stop words
stopword_list = nltk.corpus.stopwords.words('english')

In [8]:
#Instantiate CountVectorizer and fit.  Unigrams are (1,1), bigrams are (2,2).  See Feature Engineering p.46.

cv=CountVectorizer(lowercase=True, strip_accents='unicode', ngram_range=(1,1), stop_words=stopword_list)
wc_vector=cv.fit_transform(vehicle['Body'])
wc_vector.shape

(3514, 14159)

In [9]:
#View word count vector
wc_vector_dense = pd.DataFrame(wc_vector.todense(), index=vehicle.index, columns=cv.get_feature_names())
wc_vector_dense.head()

,00,000,0000,000688,000696,000697,000702,000703,000706,000lb,...,zopxhhkqe8sr,zp1629a,zp18843a,zp18944b,zr,zr1,zr17,zr19,zx2,zx4
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

#Get first 100 rows of Body series.
tokenized = vehicle['Body'].apply(word_tokenize)
tokenized.head(3)

1     [,, 2006, Ford, Focus, in, great, mechanical, ...
5     [,, ,, ,, address, :, ,, phone, :, ☎, ,, text,...
13    [,, Everything, works, ,, Turbo, diesel, ,, Ne...
Name: Body, dtype: object

In [11]:
#POS tag pandas series.  Then convert seriesPOS to table with one tuple (token, token_POS) per row.
seriesPOS = tokenized.apply(nltk.pos_tag)
adjectives = pd.DataFrame([tup for row in seriesPOS for tup in row if tup[1] == 'JJ'], columns=['tok','tok_pos'])
#adjectives = pd.DataFrame([tup for row in seriesPOS for tup in row], columns=['tok','tok_pos'])
print(adjectives.shape)

#Group combined on POS.
tok_piv = adjectives.groupby(['tok_pos','tok'])['tok'].agg({'tok':'count'})
tok_piv.nlargest(5, 'tok')

(50372, 2)


C:\Users\Justin\Anaconda3\envs\ds_env\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  


tok
tok_pos tok            
JJ      used       1743
        new        1473
        other      1237
        Subject    1018
        available   972

In [12]:
adjectives.drop_duplicates(subset=['tok'], inplace=True)
adjectives.shape

(2818, 2)

In [13]:
wcv = wc_vector_dense.transpose().reset_index()
wcv.rename(columns={"index": "token"}, inplace=True)
print(wcv.shape)
wcv.head()

(14159, 3515)


,token,1,5,13,14,17,8277,20,21,22,...,10502,10508,10509,10514,10523,10524,10527,10532,10535,10537
0,00,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
1,000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,000688,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000696,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Join adjectives and wcv on token.  Add row Total.
merged = adjectives.merge(wcv, how='inner', left_on=['tok'], right_on=['token'])
merged.drop(['tok'], axis='columns', inplace=True)
merged.loc[:, 'Total'] = merged.sum(axis=1)

In [15]:
merged.shape

(1179, 3517)

In [16]:
#Sort by most frequent adjective tokens.  Original vehicle index is in column names.
merged.sort_values(ascending=False, by='Total').head()

,tok_pos,token,1,5,13,14,17,8277,20,21,...,10508,10509,10514,10523,10524,10527,10532,10535,10537,Total
714,JJ,ford,1,3,0,0,3,0,10,1,...,9,2,1,1,1,1,9,0,1,7408
176,JJ,call,1,0,0,0,1,3,0,1,...,0,2,0,2,1,0,0,0,1,3663
22,JJ,new,0,0,2,0,1,3,1,0,...,1,0,0,0,0,0,1,0,0,3499
67,JJ,used,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,2578
205,JJ,f150,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,2518


In [17]:
pd.set_option('max_colwidth', 9000)
vehicle.loc[[5], ['Body','Model']]

,Body,Model
5,"\n ,\n,\n,\naddress: ,phone: ☎ ,text: , ,\nlink: , ,\ncontact: ,\n\n,\n,\n,after hours please text 512-761-7442,\nXL trim, Oxford White exterior and Steel interior. CARFAX 1-Owner. Flex Fuel, Tow Hitch, Edmunds.com's review says Compared to the competition, the 2015 Ford F-250 Super Duty line is remarkably quiet at highway speeds.. ,KEY FEATURES INCLUDE,Flex Fuel, Trailer Hitch. Ford XL with Oxford White exterior and Steel interior features a 8 Cylinder Engine with 385 HP at 5500 RPM*. ,EXPERTS RAVE,Edmunds.com explains Compared to the competition, the 2015 Ford F-250 Super Duty line is remarkably quiet at highway speeds.. ,WHO WE ARE,Here at Chuck Nash we do things a little different. We have been a family owned and operated dealership for over 38 years, and treat our customers as part of that family. We don't believe in pressure sales; but instead we hold the belief that if we treat our customers as we would our family, then we can create a relationship with a customer for life. We just don't want to put you into the car of your dreams and send you on your way, but we want to take care of you, and your vehicle, for years to ,Horsepower calculations based on trim engine configuration. Please confirm the accuracy of the included equipment by calling us prior to purchase. ,4-Wheel ABS., 4-Wheel Disc Brakes., 6-Speed A/T., ,8 Cylinder Engine., A/C., Adjustable Steering Wheel., ,AM/FM Stereo., Brake Assist., Driver Air Bag., ,Driver Lumbar., Electronic Stability Control., Flex Fuel., ,Front Tow Hooks., Full Size Spare Tire., Intermittent Wipers., ,Pass-Through Rear Seat., Passenger Air Bag., Passenger Vanity Mirror., ,Power Steering., Rear Head Air Bag., Rear Wheel Drive., ,Side Head Air Bag., Split Bench Seat., Steel Wheels., ,Tires - Front All-Season., Tires - Rear All-Season., Trailer Hitch., ,Vehicle Anti-Theft System., Vinyl Seats., ,Tire Pressure Monitoring System., Variable Speed Intermittent Wipers., ,\n,\naddress: ,phone: ☎ ,text: , ,\nlink: , ,\ncontact: ,\n\n,\n",f-250


### Get binned train and test

In [18]:
#Get train and test from vehicle.
X = vehicle[['Body','Price']].copy()
Y = pd.DataFrame(X.pop('Price'))  #need df for binning

from sklearn.model_selection import train_test_split
trnX, tstX, trnY, tstY = train_test_split(X, Y, test_size=0.1, random_state=42)

In [19]:
# def get_bins(train_col_df, test_col_df, bincnt = 10, binStrategy='quantile'):
#     '''Bin column dataframes and return binned arrays.'''
#     from sklearn.preprocessing import KBinsDiscretizer
#     kbd = KBinsDiscretizer(n_bins=bincnt, encode='ordinal', strategy=binStrategy)
#     trn_binned = kbd.fit_transform(train_col_df)
#     tst_binned = kbd.transform(test_col_df)
#     return trn_binned, tst_binned
# trn_binned, tst_binned = get_bins(trnY, tstY)

# #Bin histogram for strategy.
# plt.hist(trn_binned)
# plt.show()

### TFIDF scores for X

In [20]:
#Get tf-idf scores for docs.
 
#Instantiate.  Use nltk stop words from above.  strip_accents='unicode' improves slightly.
tf=TfidfVectorizer(stop_words=None, smooth_idf=True, use_idf=True, vocabulary=None, strip_accents='unicode')
 
#Learn vocabulary and idf.  Return term-document matrix for train and test.
trn_vect=tf.fit_transform(trnX['Body'])  #must be series, NOT dataframe
tst_vect=tf.transform(tstX['Body'])  #must be series, NOT dataframe
print('trn_vect shape --', trn_vect.shape)
print('tst_vect shape --', tst_vect.shape)

 
#Put tf-idf values in df.  Rarest are: 260 and gasoline.
df=pd.DataFrame(trn_vect.T.todense(), index=tf.get_feature_names(), columns=trnX.index)
print('\ntrn_vect:')
display(df.head())

#Model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
rr=Ridge(alpha=0.5)
rr.fit(trn_vect, trnY)
pred_y = rr.predict(tst_vect)

#Score
score_algo(rr, tst_vect, tstY, pred_y)

trn_vect shape -- (3162, 13451)
tst_vect shape -- (352, 13451)

trn_vect:


,563,855,3906,4679,9823,2904,10164,4230,5814,3085,...,1197,6088,4351,8863,2795,2878,3220,2151,10514,9153
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.045771,...,0.044521,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014207,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.017655,0.0,0.0,0.0,0.0
0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
000696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
000697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


-----------------------------------------------
##### Ridge - Prediction Scores:
-----------------------------------------------
RMSE: 7833.3
 R^2: 0.4783
 MAE: 5676.2


### Get word frequencies with TfidfTransformer

In [21]:
#Instantiate.  Use nltk stop words from above.
cv=CountVectorizer(stop_words=stopword_list)
trn_vect=cv.fit_transform(trnX['Body'])  #must be series, NOT dataframe
tst_vect=cv.transform(tstX['Body'])  #must be series, NOT dataframe
print('trn_vect shape --', trn_vect.shape)
print('tst_vect shape --', tst_vect.shape)

#View one-hot word count vector
display(pd.DataFrame(trn_vect.todense(), columns=cv.get_feature_names()).head())

#Compute DF values
tt=TfidfTransformer(use_idf=False)

trn_vect2=tt.fit_transform(trn_vect)  #must be series, NOT dataframe
tst_vect2=tt.transform(tst_vect)  #must be series, NOT dataframe
print('trn_vect shape --', trn_vect2.shape)
print('tst_vect shape --', tst_vect2.shape)

#View tfidtransformer result
print('\ntrn_vect2:')
df2=pd.DataFrame(trn_vect2.todense(), columns=cv.get_feature_names())
display(df2.head())

#Model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
rr=Ridge(alpha=0.5)
rr.fit(trn_vect2, trnY)
pred_y = rr.predict(tst_vect2)

#Score
score_algo(rr, tst_vect, tstY, pred_y)

trn_vect shape -- (3162, 13354)
tst_vect shape -- (352, 13354)


,00,000,0000,000696,000697,000702,000703,000lb,000mi,000miles,...,zr,zr1,zr17,zr19,zx2,zx4,él,éxito,último,único
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


trn_vect shape -- (3162, 13354)
tst_vect shape -- (352, 13354)

trn_vect2:


,00,000,0000,000696,000697,000702,000703,000lb,000mi,000miles,...,zr,zr1,zr17,zr19,zx2,zx4,él,éxito,último,único
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


-----------------------------------------------
##### Ridge - Prediction Scores:
-----------------------------------------------
RMSE: 7906.2
 R^2: 0.4685
 MAE: 5709.0


###  One-Feature Ridge Model

In [22]:
train=pd.read_sql('SELECT * from "trainClean"', engine, index_col='index')
test=pd.read_sql('SELECT * from "testClean"', engine, index_col='index')

#One feature model
from sklearn.linear_model import Ridge
rr2=Ridge(alpha=0.01, solver='cholesky')
rr2.fit(train[['Year^2']], train['Price'])
pred2_y = rr2.predict(test[['Year^2']])

#Score
score_algo(rr2, test[['Year^2']], test['Price'], pred2_y)

-----------------------------------------------
##### Ridge - Prediction Scores:
-----------------------------------------------
RMSE: 9062.8
 R^2: 0.2897
 MAE: 6904.2


### Something to read: A Practitioner's Guide to Natural Language Processing
###### https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72